In [1]:
import pandas as pd
import numpy as np

In [2]:
DIR = '../dataset/'
DATASET = 'acs_5yr_est_selected_economic_characteristics_2010-2022.csv'
df = pd.read_csv(DIR + DATASET)

Replace (X) with empty


In [3]:
df.columns = ['label', 'category', 'state', 'estimate', 'moe', 'percent', 'percent moe', 'year']
df = df.replace("(X)", "")
df['percent'] = df['percent'].replace("%", "")

Strip and lower text rows

In [4]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip().str.lower()

Working with employment status only

In [5]:
filtered_df = df[df['label'] == 'unemployed']
drop_cols = ['label', 'moe', 'percent moe', 'category', 'percent']
filtered_df = filtered_df.drop(drop_cols, axis=1)
filtered_df

,state,estimate,year
5,alabama,"194,012",2010
150,alaska,"31,231",2010
295,arizona,"227,691",2010
440,arkansas,"106,798",2010
585,california,"1,642,405",2010
...,...,...,...
97300,washington,"198,259",2021
97445,west virginia,"50,277",2021
97590,wisconsin,"109,017",2021
97735,wyoming,"12,464",2021


Add % total unemployment to numbers

In [6]:
df_percent_unemployed = df[df['label'].isin(['percent unemployed', 'unemployment rate'])]
keep_cols = ['state', 'percent', 'year']
df_percent_unemployed = df_percent_unemployed[keep_cols]
df_percent_unemployed['percent'] = df_percent_unemployed['percent'].str.replace("%", "")
df_percent_unemployed['percent'] = df_percent_unemployed['percent'].astype(float)
df_percent_unemployed
filtered_df = pd.merge(filtered_df, df_percent_unemployed, on=['state', 'year'], how='left')
filtered_df = filtered_df.rename(columns={'percent': 'labour_force_percent'})
filtered_df

,state,estimate,year,labour_force_percent
0,alabama,"194,012",2010,8.7
1,alaska,"31,231",2010,8.6
2,arizona,"227,691",2010,7.7
3,arkansas,"106,798",2010,7.8
4,california,"1,642,405",2010,9.0
...,...,...,...,...
671,washington,"198,259",2021,5.1
672,west virginia,"50,277",2021,6.4
673,wisconsin,"109,017",2021,3.5
674,wyoming,"12,464",2021,4.2


Check missing data

In [7]:
missing = filtered_df.isnull().sum()
missing

state                   0
estimate                0
year                    0
labour_force_percent    0
dtype: int64

Sort by year then add to csv

In [8]:
filtered_df = filtered_df.sort_values(by=['state', 'year'])
filtered_df

,state,estimate,year,labour_force_percent
0,alabama,"194,012",2010,8.7
52,alabama,"215,543",2011,9.6
156,alabama,"230,778",2012,10.3
104,alabama,"241,930",2013,10.8
364,alabama,"228,716",2014,10.2
...,...,...,...,...
518,wyoming,"13,685",2018,4.5
466,wyoming,"13,544",2019,4.5
622,wyoming,"13,222",2020,4.4
674,wyoming,"12,464",2021,4.2


In [10]:
filtered_df.to_csv(DIR + 'unemployment.csv', index=False)